
# 1. Imports

In [0]:
from pyspark.sql.functions import (
    from_utc_timestamp,
    date_format,
    dayofweek,
    sum
)

from pyspark.sql.types import (
    StringType,
)


# 2. Carregando sparkDFs


## 2.1. Total payment order

In [0]:
# Local do arquivo Delta
path_to_delta_table = "dbfs:/FileStore/Datum/KaggleOlistData/silver/delta/order_payments"

# Ler o arquivo Delta como um DataFrame
df_order_payments = spark.read.format("delta").load(path_to_delta_table)

In [0]:
display(df_order_payments.take(10))

order_id,payment_sequential,payment_type,payment_installments,payment_value
b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.330000000000000000
a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.390000000000000000
25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.710000000000000000
ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.780000000000000000
42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.450000000000000000
298fcdf1f73eb413e4d26d01b25bc1cd,1,credit_card,2,96.120000000000000000
771ee386b001f06208a7419e4fc1bbd7,1,credit_card,1,81.160000000000000000
3d7239c394a212faae122962df514ac7,1,credit_card,3,51.840000000000000000
1f78449c87a54faf9e96e88ba1491fa9,1,credit_card,6,341.090000000000000000
0573b5e23cbd798006520e1d5b4c6714,1,boleto,1,51.950000000000000000



## 2.2 Orders

In [0]:
# Local do arquivo Delta
path_to_delta_table = "dbfs:/FileStore/Datum/KaggleOlistData/silver/delta/orders"

# Ler o arquivo Delta como um DataFrame
df_orders = spark.read.format("delta").load(path_to_delta_table)

In [0]:
display(df_orders.take(10))

order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_diff
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02T10:56:33Z,2017-10-02T11:07:15Z,2017-10-04T19:55:00Z,2017-10-10T21:25:13Z,2017-10-18T00:00:00Z,-8
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24T20:41:37Z,2018-07-26T03:24:27Z,2018-07-26T14:31:00Z,2018-08-07T15:27:45Z,2018-08-13T00:00:00Z,-6
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08T08:38:49Z,2018-08-08T08:55:23Z,2018-08-08T13:50:00Z,2018-08-17T18:06:29Z,2018-09-04T00:00:00Z,-18
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18T19:28:06Z,2017-11-18T19:45:59Z,2017-11-22T13:39:59Z,2017-12-02T00:28:42Z,2017-12-15T00:00:00Z,-13
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13T21:18:39Z,2018-02-13T22:20:29Z,2018-02-14T19:46:34Z,2018-02-16T18:17:02Z,2018-02-26T00:00:00Z,-10
a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09T21:57:05Z,2017-07-09T22:10:13Z,2017-07-11T14:58:04Z,2017-07-26T10:57:55Z,2017-08-01T00:00:00Z,-6
136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11T12:22:08Z,2017-04-13T13:25:17Z,null,null,2017-05-09T00:00:00Z,null
6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16T13:10:30Z,2017-05-16T13:22:11Z,2017-05-22T10:07:46Z,2017-05-26T12:55:51Z,2017-06-07T00:00:00Z,-12
76c6e866289321a7c93b82b54852dc33,f54a9f0e6b351c431402b8461ea51999,delivered,2017-01-23T18:29:09Z,2017-01-25T02:50:47Z,2017-01-26T14:16:31Z,2017-02-02T14:08:10Z,2017-03-06T00:00:00Z,-32
e69bfb5eb88e0ed6a785585b27e16dbf,31ad1d1b63eb9962463f764d4e6e0c9d,delivered,2017-07-29T11:55:02Z,2017-07-29T12:05:32Z,2017-08-10T19:45:24Z,2017-08-16T17:14:30Z,2017-08-23T00:00:00Z,-7


In [0]:
del path_to_delta_table


# 3. Identificando Padrões de Compra

> Teste Prático -> Introduza uma regra mais complexa, como identificar padrões de comportamento de compra ao longo do tempo ou criar categorias personalizadas de produtos com base em determinados critérios.


## 3.1 Identificação do dia da semana, por mês, que gera o maior valor de venda

In [0]:
df_total_payment_order = df_order_payments.groupBy("order_id").agg(
    sum("payment_value").alias("total_payment_value")
)

In [0]:
df_total_payment_order_with_weekday = df_total_payment_order.join(
    df_orders,
    df_total_payment_order.order_id == df_orders.order_id,
    "left",
).select(
    df_total_payment_order["*"],
    df_orders["customer_id"],
    df_orders["order_purchase_timestamp"],
)

In [0]:
# Criando coluna com em BRT Timezone
df_total_payment_order_with_weekday = df_total_payment_order_with_weekday.withColumn(
    "brt_timestamp", from_utc_timestamp("order_purchase_timestamp", "America/Sao_Paulo")
)

# "Retirando" o Z (Zulu Time), do brt_timestamp, para não gerar confução na interpretação do timestamp:
df_total_payment_order_with_weekday = df_total_payment_order_with_weekday.withColumn(
    "brt_formatted", date_format("brt_timestamp", "yyyy-MM-dd HH:mm:ss")
)

# Criando coluna com o dia da semana como número
df_total_payment_order_with_weekday = df_total_payment_order_with_weekday.withColumn(
    "day_of_week_num", dayofweek("brt_formatted")
)

# Criando coluna com o dia da semana como nome
df_total_payment_order_with_weekday = df_total_payment_order_with_weekday.withColumn(
    "day_of_week_name", date_format("brt_formatted", "EEEE")
)

# Selecionando e renomeando as colunas desejadas
df_total_payment_order_with_weekday = df_total_payment_order_with_weekday.select(
    "order_id",
    "total_payment_value",
    "customer_id",
    df_total_payment_order_with_weekday.brt_formatted.alias(
        "order_purchase_timestamp_brt"
    ),
    "day_of_week_num",
    "day_of_week_name",
)

In [0]:
display(df_total_payment_order_with_weekday.take(10))

order_id,total_payment_value,customer_id,order_purchase_timestamp_brt,day_of_week_num,day_of_week_name
bb2d7e3141540afc268df7ef6580fc75,37.150000000000000000,8b325e34eab50c4b1e54500a3ff31e6e,2018-08-09 13:28:34,5,Thursday
85be7c94bcd3f908fc877157ee21f755,72.750000000000000000,54b307e9a6121f572faef4f220fbac3a,2017-09-21 19:00:36,5,Thursday
8ca5bdac5ebe8f2d6fc9171d5ebc906a,189.080000000000000000,155716e7d7dbee88852ff931b86889aa,2018-03-18 19:58:56,1,Sunday
54066aeaaf3ac32e7bb6e45aa3bf65e4,148.060000000000000000,6a0fdc8625275b516a4cf721d28261f8,2017-06-01 08:22:39,5,Thursday
5db54d41d5ebd6d76cb69223355601f5,136.260000000000000000,fe66ded86a1678a8ad544e4c78447348,2017-03-02 15:53:45,5,Thursday
41537821ce113ccef16d0e4b303f1230,89.270000000000000000,0312590499e722678eeb292f4da77e77,2017-12-11 11:56:03,2,Monday
beca5b5e9460824d84352c01cb89b421,239.500000000000000000,18c39fea8fbb5612f4e5b8bbecf60d5b,2018-01-30 09:03:14,3,Tuesday
33f1e992ba3e439bfd0d432164a3d44a,248.510000000000000000,ef898057565789279a822bd22a0b6dc0,2018-03-01 09:57:42,5,Thursday
3fa59277573f0fe06ece044ebec2eb02,91.050000000000000000,ee1fb534e64d33fcb16a1023dc721a24,2018-01-12 19:59:20,6,Friday
e239d280236cdd3c40cb2c033f681d1c,102.030000000000000000,8f3b9d3f52f2f606b6675ef3d7d85e7b,2018-03-01 15:32:58,5,Thursday


In [0]:
df_total_payment_sum_per_weekday = df_total_payment_order_with_weekday.groupBy(
    "day_of_week_name"
).agg(sum("total_payment_value"))

In [0]:
display(df_total_payment_sum_per_weekday.take(10))

day_of_week_name,sum(total_payment_value)
Wednesday,2481998.830000000000000000
Tuesday,2580433.480000000000000000
Friday,2268808.260000000000000000
Thursday,2409388.460000000000000000
Saturday,1766722.480000000000000000
Monday,2629227.920000000000000000
Sunday,1872292.690000000000000000


In [0]:
# cast timestamp em order_purchase_timestamp_brt
df_total_payment_order_with_weekday = df_total_payment_order_with_weekday.withColumn(
    "order_purchase_timestamp_brt",
    df_total_payment_order_with_weekday["order_purchase_timestamp_brt"].cast(
        "timestamp"
    ),
)

# Adicionando uma coluna para ano e mês no formato 'yyyy-MM'
df_total_payment_order_with_weekday = df_total_payment_order_with_weekday.withColumn(
    "year_month", date_format("order_purchase_timestamp_brt", "yyyy-MM")
)

# Agrupando por ano-mês e dia da semana e calculando a soma
df_total_sales_by_month_by_day = (
    df_total_payment_order_with_weekday.groupBy("year_month", "day_of_week_name")
    .agg(sum("total_payment_value").alias("total_sales"))
    .orderBy("year_month", "total_sales", ascending=[0, 0])
)

In [0]:
display(df_total_sales_by_month_by_day.take(10))

year_month,day_of_week_name,total_sales
2018-10,Wednesday,287.260000000000000000
2018-10,Tuesday,222.030000000000000000
2018-10,Monday,80.380000000000000000
2018-09,Thursday,3048.700000000000000000
2018-09,Monday,594.390000000000000000
2018-09,Tuesday,480.210000000000000000
2018-09,Wednesday,179.210000000000000000
2018-09,Saturday,137.030000000000000000
2018-08,Wednesday,178487.660000000000000000
2018-08,Monday,162395.600000000000000000


In [0]:
# Função de tradução de inglês para português
def translate_weekday(weekday):
    translations = {
        "Monday": "Segunda-feira",
        "Tuesday": "Terça-feira",
        "Wednesday": "Quarta-feira",
        "Thursday": "Quinta-feira",
        "Friday": "Sexta-feira",
        "Saturday": "Sábado",
        "Sunday": "Domingo",
    }
    return translations.get(weekday, weekday)


# User-Defined Function
translate_weekday_udf = udf(translate_weekday, StringType())

In [0]:
# Aplicando a tradução à coluna "day_of_week_name"
df_total_sales_by_month_by_day = df_total_sales_by_month_by_day.withColumn(
    "day_of_week_name", translate_weekday_udf(df_total_sales_by_month_by_day["day_of_week_name"])
)

In [0]:
display(df_total_sales_by_month_by_day.take(10))

year_month,day_of_week_name,total_sales
2018-10,Quarta-feira,287.260000000000000000
2018-10,Terça-feira,222.030000000000000000
2018-10,Segunda-feira,80.380000000000000000
2018-09,Quinta-feira,3048.700000000000000000
2018-09,Segunda-feira,594.390000000000000000
2018-09,Terça-feira,480.210000000000000000
2018-09,Quarta-feira,179.210000000000000000
2018-09,Sábado,137.030000000000000000
2018-08,Quarta-feira,178487.660000000000000000
2018-08,Segunda-feira,162395.600000000000000000


In [0]:
del df_order_payments, df_orders, df_total_payment_order, df_total_payment_order_with_weekday, df_total_payment_sum_per_weekday


# 4. Exportando dados para Delta Lake

> Teste Prático -> "Grave os mesmos dados em formato Delta Lake para aproveitar as funcionalidades de versionamento e transações ACID."

In [0]:
df_total_sales_by_month_by_day.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(
    "dbfs:/FileStore/Datum/KaggleOlistData/gold/delta/total_sales_by_month_by_day"
)

In [0]:
del df_total_sales_by_month_by_day


# 5. Criando Delta Table

In [0]:
%sql

create database if not exists olist;

In [0]:
%sql

use olist;

In [0]:
%sql

drop table if exists total_sales_by_month_by_day;

In [0]:
%sql

CREATE TABLE total_sales_by_month_by_day
USING DELTA
LOCATION "dbfs:/FileStore/Datum/KaggleOlistData/gold/delta/total_sales_by_month_by_day"